In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile
import shutil
import numpy as np 
# linear algebra
import pandas as pd

In [4]:
train_csv=pd.read_csv("train_dance.csv")
train_csv['target'].value_counts()

mohiniyattam     50
odissi           49
kathakali        47
bharatanatyam    47
kuchipudi        46
sattriya         45
kathak           44
manipuri         36
Name: target, dtype: int64

In [5]:

X=train_csv['Image']
y=train_csv['target']

In [10]:

os.mkdir('final_train_dir')
for i in train_csv['target'].unique():
    os.mkdir('final_train_dir/'+i)

In [15]:
for i in train_csv['target'].unique():
    for j in X[y==i]:
        copyfile('Downloads/dance/train/'+j, 'final_train_dir/'+i+'/'+j)

In [16]:
os.listdir('final_train_dir')

['bharatanatyam',
 'kathak',
 'kathakali',
 'kuchipudi',
 'manipuri',
 'mohiniyattam',
 'odissi',
 'sattriya']

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [18]:
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                        factor=0.1,
                                        patience=2,
                                        cooldown=2,
                                        min_lr=0.00001,
                                        verbose=1)

callbacks = [reduce_learning_rate]

In [19]:

vggmodel =VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3),pooling='max')

vggmodel.trainable = False
model = Sequential([
  vggmodel, 
  Dense(1024, activation='relu'),
  Dropout(0.15),
  Dense(256, activation='relu'),
  Dropout(0.15),
  Dense(8, activation='softmax'),
])

model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
train_datagenerator = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range=40,  
        zoom_range = 0.20,  
        width_shift_range=0.10,  
        height_shift_range=0.10,  
        horizontal_flip=True,  
        vertical_flip=False) 

image_size=224
train_generator=train_datagenerator.flow_from_directory(
        r"final_train_dir",
        target_size=(image_size,image_size),
#        batch_size=128,
        class_mode='categorical'
        )

Found 364 images belonging to 8 classes.


In [24]:
history =model.fit_generator(
    train_generator,
    verbose=1,
    epochs=40,
   callbacks=callbacks
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 12 steps
Epoch 1/40
12/12 [==============================] - 77s 6s/step - loss: 2.4806 - accuracy: 0.1346
Epoch 2/40
12/12 [==============================] - 82s 7s/step - loss: 1.8291 - accuracy: 0.3269
Epoch 3/40
12/12 [==============================] - 88s 7s/step - loss: 1.5958 - accuracy: 0.4038
Epoch 4/40
12/12 [==============================] - 90s 7s/step - loss: 1.4579 - accuracy: 0.4478
Epoch 5/40
12/12 [==============================] - 86s 7s/step - loss: 1.2387 - accuracy: 0.5549
Epoch 6/40
12/12 [==============================] - 86s 7s/step - loss: 1.1558 - accuracy: 0.5907
Epoch 7/40
12/12 [==============================] - 91s 8s/step - loss: 1.0830 - accuracy: 0.6566
Epoch 8/40
12/12 [==============================] - 90s 8s/step - loss: 1.0570 - accuracy: 0.6181
Epoch 9/40
12/12 [==============================] - 96s 8s/step - loss: 1.0311 - accuracy: 0.637

In [25]:
shutil.copytree('Downloads/dance/test', 'test/images')

'test/images'

In [27]:
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator=test_datagen.flow_from_directory(
        r'test',
        target_size=(image_size,image_size),
#       color_mode="rgb",
        batch_size=32,
        class_mode=None,
        shuffle=False
        )

Found 156 images belonging to 1 classes.


In [30]:

pred=model.predict_generator(test_generator,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)

labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results["Filename"]=results["Filename"].apply(lambda x:x[7:])

test_csv=pd.read_csv("test.csv")

results.set_index(["Filename"],inplace=True)
test_csv.set_index(["Image"],inplace=True)

output=test_csv.merge(results,left_index=True,right_index=True)
output.index.name='Image'
output.rename(columns={'Predictions':'target'},inplace=True)
output.to_csv('submission.csv')


Instructions for updating:
Please use Model.predict, which supports generators.
5/5 [==============================] - 39s 8s/step
